<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [47]:
import pandas as pd
import re
import requests
import numpy as np

class CMAP:
    """
    This is a class to retrieve, store, and post cmaps in cmap servers.
    It is also capable of enumerating parts of specific cmaps.
    TODO: 
    1) Add a fileRead method to read cxl passed in as a file instead of a string
    2) Vectorize some of the iterable loops to make them more efficient
    3) Move subDic2DF to separate library and import
    4) Move to_xml to separate library and import
    5) (maybe) Develop a dictionary of column names, and the dfs that use each column name
    6) add resMetaPeople per proposition and per concept
    
    
    """
    def __init__ (self, \
             cmapID = None, \
             baseURL = 'https://cmapscloud.ihmc.us/resources/rid=', \
             authEmail ='ic3foods@gmail.com', \
             cxlDoc=None):
        #Make sure that both an ID and a CXL were not passed simultaneously:
        if cxlDoc is not None and cmapID is not None:
            raise ValueError("""
            You submitted both a cxlDoc file and a CMAPID.
            We don't know which to parse.
            Perhaps you meant to POST the cxlDoc to a CMAP Server?
            """)
        #Make some basic variables
        self.baseURL = baseURL
        self.authEmail = authEmail
        self.cxlDoc = cxlDoc
        self.itemCounter = None
        self.dicResMetaSansPeople = {}
        self.dicResMetaPeople = {}
        self.dicMap = {}
        self.dicMap['concept-appearance'] = {}
        self.dicMap['linking-phrase-appearance'] = {}
        self.dicMap['resource-group-list'] = {}
        self.dicMap['resource-group-list']['resource-group'] = {}
        self.dicMap['style-sheet-list'] = {}
        self.dicMap['style-sheet-list']['style-sheet'] = {}
        self.dicMap['extra-property-list'] = {}
        self.dicMap['extra-graphical-property-list'] = {}
        self.listResMetaPeopleOrgs = []
        self.listNodeApprncs = [self.dicMap['concept-appearance'],self.dicMap['linking-phrase-appearance']]
        self.listResourcePropTypes = [self.dicMap['extra-property-list'],self.dicMap['extra-graphical-property-list']]
        self.rootDicName = None
        self.rootDicVal = None 
        #Make empty dfs for filling from db or after parse
        self.dfResMetaList = pd.DataFrame(columns=[			
			#Name							Type	Req	Description
            'count',				#	unsignedInt	No	Total number of children currently in the list
            'total-count',			#	unsignedInt	No	Total number of children in the source data set
            'start-index',			#	unsignedInt	No	Index within the source data set of the first child in the list
            'end-index',			#	unsignedInt	No	Index within the source data set of the last child in the list
            'sort-as',				#	string	No	alpha if sorted alphabetically, or numeric if sorted numerically
            'sort-dir',				#	string	No	Direction of the sort, either ascending or descending
            'sort-by',				#	string	No	XPath expression identifying the element or attribute compared for sorting
                                                ])
        self.dfPartList = pd.DataFrame(columns=[			
			#Name							Type	Req	Description            
            'count',				#	unsignedInt	No	Total number of children currently in the list
            'total-count',			#	unsignedInt	No	Total number of children in the source data set
            'start-index',			#	unsignedInt	No	Index within the source data set of the first child in the list
            'end-index',			#	unsignedInt	No	Index within the source data set of the last child in the list
            'sort-as',				#	string	No	alpha if sorted alphabetically, or numeric if sorted numerically
            'sort-dir',				#	string	No	Direction of the sort, either ascending or descending
            'sort-by',    #	string	No	XPath expression identifying the element or attribute compared for sorting
                                                ])
        self.dfParts = pd.DataFrame(columns=[			
            #Name					#  Min. Max. Note  
            'content-type'			#	0	1	Attribute: If the referenced resource is a container then this attribute will tell 
            									#what type of elements it contains inside the content element. 
            									#It can either be res-meta-list or part-list
            'dc:format'				#	0	1	#MIME type of the resource.
            'dc:identifier'			#	0	1	#HTTP URL with resource id (e.g. http://cmap.ihmc.us/rid=10002929_292992_19)
            'dc:relation'			#	0	1	#HTTP URL, location based (e.g. http://cmap.ihmc.us/plants.cxl)
                                                ])
        self.dfResMetaSansPeople = pd.DataFrame(columns=[	#Resource metadaata, but without the people, who have been extracted to a different df
            #dublin core namespaces	#  Min. Max. Note  
            'content-type',			#	0	1	Attribute: If the referenced resource is a container then this attribute 
                                    #			will tell what type of elements it contains inside the content element. 
                                    #			It can either be res-meta-list or part-list
            #'dc:contributor',		#	0	1	moved to people table
            #'dc:creator',			#	0	1	moved to people table
            'mapID',				#			the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'dc:description',		#	0	1	also called "focus question"
            'dc:format',			#	0	1	'x-cmap/x-storable'
            'dc:identifier',		#	0		HTTP URL with resource id (e.g. http://cmap.ihmc.us/rid=10002929_292992_19)
            'dc:language',			#	0		Language code (RFC1766) (e.g. en_US)
            'dc:publisher',			#	0	1	e.g. IHMC CmapTools v. 4.0
            'dc:relation',			#	0	1	HTTP URL, location based (e.g. http://cmap.ihmc.us/plants.cxl)
            'dc:source',			#	0	1	cmap:<server-id>:<folder-id>:<resource-id>
            'dc:subject',			#	0	1	keyword one,keyword 2
            'dc:title',				#	0	1	Name of the Cmap
            'dcterms:created',		#	0	1	date created
            'dcterms:extent',		#	0	1	size in bytes
            'dcterms:modified',		#	0	1	date last modified
            #'dcterms:rightsHolder',#	0	1	moved to people table
                                                ])	
        self.dfResMetaPeople = pd.DataFrame(columns=[		#People removed from the resource metadata
            #'Name',				Type		Req	Description	
            'mapID',				#string			the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'propID',				#string		No	FUTURE:the ID of the proposition, for assigning credit to who made it, null if for whole map or for concept
            'conceptID',			#string		No	FUTURE:the ID of the concept, for assigning credit to who made it, null if for whole map or for proposition
            'role',					#string		No	the role of the person, one of: ['creator', 'contributor', 'rightsHolder']
            'vcard:FN',				#string		No	full name
            'vcard:EMAIL',			#string		No	email address
            'vcard:ORG',			#string		No	organization name
                                                ])		
        self.dfMap = pd.DataFrame(columns=[					#Map = a collection of concepts, their named/enumerated connections to each other, w vis details
            #'Name',				Type		Req	Description	
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'root-id',				#string		No	The id of the root concept of the map
            'header',				#string		No	The text of the header
            'footer',				#string		No	The text of the footer
            'width',				#int		No	the minimum width of the map's canvas
            'height',				#int		No	the minimum height of the map's canvas
            'default-stylesheet-id',#string		No	the unique id of the maps current default named stylesheet
            'weight',				#decimal	No	Predefined weight associated with this Node. (Not Supported by CmapTools)
                                                ])
        self.dfConcepts = pd.DataFrame(columns=[			#Concept = "Subject/Object" = "Node" depending on the context
            #'Name',				Type		Req	Description	
            'mapID',				#string			the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#string		No	The unique Id of the Node. Must be unique for all nodes concepts and linking phrases
            'parent-id',			#string		No	The unique Id of the parent container if not the map (usually the Id of another Node)
            'label',				#string		Yes	The text displayed in the node on screen. Used as the reference id if no id is provided
            'short-comment',		#string		No	Extra concise text info about this node: THIS IS MOUSE-OVER INFO IN CMAPS
            'long-comment',			#string		No	Extra text info about this node: THIS IS HIDDEN, BUT SEARCHABLE TEXT IN CMAPS
            'uri',					#string		No	Unique identifier for this entity
            'weight',				#decimal	No	Predefined weight associated with this Node. (Not Supported by CmapTools)
                                                ])
        self.dfConceptApprncs = pd.DataFrame(columns=[		#Defines the individual appearance information for a concept
            #'Name',				#	Type	Req	Description	
            'mapID',				#string			the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string	Yes	the unique id of the concept to which this appearance is applied	
            'x',					#	int		No	the center x location of this node (default is 10)	
            'y',					#	int		No	the center y location of this node (default is 10)	
            'width',				#	int		No	"The width in pixels of the bounding box as computed by the last system to render the map.
													#This is a suggested starting value and will be recomputed (by CMAPs) when rendering the node using
													#the text-margin, min-width, background-image, font information, and border-thickness attributes"	
            'height',				#	int	No	"The height in pixels of the bounding box as computed by the last system to render the map.
													#This is a suggested starting value and will be recomputed (by CMAPs) when rendering the node using
													#the text-margin, min-width, background-image, font information, and border-thickness attributes"	
            'min-width',			#	int		No	the minimum width in pixels.	
            'min-height',			#	int		No	the minimum height in pixels.	
            'expanded',				#	boolean	No	if true and this is a nested node then it should be drawn as expanded (default is false)	
            'draw-order',			#	int		No	index into draw order relative to parent (z-order)	
            'font-name',			#	string	No	the font family's name	
            'font-size',			#	decimal	No	the font point size	
            'font-style',			#	string	No	the font style (<plain>|italic|bold|underlined)	
            'font-color',			#	string	No	font color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])	
            'text-margin',			#	int		No	size of whitespace around node text in pixels	
            'text-alignment',		#	string	No	text alignment in node bounds (<center>, top, bottom, left, right, top-left, top-right, bottom-left, bottom-right)	
            'background-color',		#	string	No	background color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])	
            'background-image',		#	string	No	the background image id reference. If "none" then no background image	
            'background-image-style',# 	string	No	How to draw the background image (<full>, scaled, cropped, or tiled)	
            'border-color',			#  	string	No	border color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])	
            'border-style',			#	string	No	border line style (none, <solid>, dotted, dashed, dash-dot, dash-dot-dot)	
            'border-thickness',		#  	int		No	border line thickness (0-6 where 0 = no border and 6 is the largest)	
            'border-shape',			#	string	No	the border shape (rectangle, <rounded-rectangle>, or oval)	
            'shadow-color',			#	string	No	shadow color (an RGBA value [0-255, 0-255, 0-255, 0 or 255] or "none" if no shadow)	
            'stylesheet-id',		#	string	No	style info to apply to this node (default is _default_)
                                                ])
        self.dfLinkingPhrases = pd.DataFrame(columns=[		#LinkPhrases = "Predicates" = "Relations" = "EdgeLabels" depending on the context
															#Note: LinkPhrase does not contain directionality, must get Connection in/out for that
            #'Name',				#	 Type	Req	Description
            'mapID',				#string			the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string	No	The unique Id of the Node. Must be unique for all nodes concepts and linking phrases
            'parent-id',			#	string	No	The unique Id of the parent container if not the map (usually the Id of another Node)
            'label',				#	string	Yes	The text displayed in the node on screen. Used as the reference id if no id is provided
            'short-comment',		#	string	No	Extra concise text info about this node: THIS IS MOUSE-OVER INFO IN CMAPS
            'long-comment',			#	string	No	Extra text info about this node: THIS IS HIDDEN, BUT SEARCHABLE TEXT IN CMAPS
            'uri',					#	string	No	Unique identifier for this entity
                                                ])
        self.dfLinkingPhraseApprncs = pd.DataFrame(columns=[	#Defines the individual appearance information for a linking phrase		
            #'Name',				#	Type	Req	Description	
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string	Yes	the unique id of the linking phrase to which this appearance is applied
            'x',					#	int		No	the center x location of this node (default is 10)
            'y',					#	int		No	the center y location of this node (default is 10)
            'width',				#	int		No	"The width in pixels of the bounding box as computed by the last system to render the map.
													#This is a suggested starting value and will be recomputed (by CMAPs) when rendering the node using
													#the text-margin, min-width, background-image, font information, and border-thickness attributes"
            'height',				#	int		No	"The height in pixels of the bounding box as computed by the last system to render the map.
													#This is a suggested starting value and will be recomputed (by CMAPs) when rendering the node using
													#the text-margin, min-width, background-image, font information, and border-thickness attributes"
            'min-width',			#	int		No	the minimum width in pixels.
            'min-height',			#	int		No	the minimum height in pixels.
            'expanded',				#	boolean	No	if true and this is a nested node then it should be drawn as expanded (default is false)
            'draw-order',			#	int			No	index into draw order relative to parent (z-order)
            'font-name',			#	string	No	the font family's name
            'font-size', 			#	decimal	No	the font point size
            'font-style',			#	string	No	the font style (<plain>|italic|bold|underlined)
            'font-color',			#	string	No	font color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'text-margin',			#	int		No	size of whitespace around node text in pixels
            'text-alignment',		#	string	No	text alignment in node bounds (<center>, top, bottom, left, right, top-left, top-right, bottom-left, bottom-right)
            'background-color',		#	string	No	background color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'background-image',		#	string	No	the background image id reference. If "none" then no background image
            'background-image-style',#	string	No	How to draw the background image (<full>, scaled, cropped, or tiled)
            'border-color',			#	string	No	border color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'border-style',			#	string	No	border line style (none, <solid>, dotted, dashed, dash-dot, dash-dot-dot)
            'border-thickness',		#	int		No	border line thickness (0-6 where 0 = no border and 6 is the largest)
            'border-shape',			#	string	No	the border shape (rectangle, <rounded-rectangle>, or oval)
            'shadow-color',			#	string	No	shadow color (an RGBA value [0-255, 0-255, 0-255, 0 or 255] or "none" if no shadow)
            'stylesheet-id', 		#	string	No	style info to apply to this node (default is _default_)
                                                            ])			
        self.dfLocalizedStyles = pd.DataFrame(columns=[		#Localized styles are applied to the text in the specified begin/end region if begin is not specified it is assumed to be 0, and if end is not specified it is assumed to be the end of the text. So by not setting begin and end you can apply the style change to the whole text. Order matters and if localized style bounds overlap the last specified will win.		
            #'Name',				#	Type	Req	Description	
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'nodeID',				#	string		the ID of the Node (concept of linkPhrase) where the specified style overrides the default : NOT IN CMAPs!
            'begin',				#	int		No	start index into the node's text where the specified change overrides the default
            'end',					#	int		No	end index into the node's text where the specified change overrides the default (end not included)
            'font-name', 			#	string	No	Localized font name
            'font-size',			#	decimal	No	Localized font point size
            'font-style',			#	string	No	Localized font style (<plain>|italic|bold|underlined)
            'font-color',			#	string	No	Localized font color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
                                                            ])			
        self.dfConnections = pd.DataFrame(columns=[			#The connections in the map
            #Name						Type	Req	Description
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string	No	The unique Id of the Connection (required only if appearance is defined)
            'parent-id',			#	string	No	The unique Id of the parent container if not the map (usually the Id of another Node)
            'from-id',				#	string	Yes	The unique Id of the Node where the connection starts
            'to-id',				#	string	Yes	The unique Id of the Node where the connection ends
            'isBidirectional',		#	boolean	No	true if the connection is bidirectional (defaults to false)
                                                            ])			
        self.dfConnectionApprncs = pd.DataFrame(columns=[	#Defines the individual appearance information for a connection		
            #'Name',				#	Type	Req	Description	
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string	Yes	the unique id of the connection to which this appearance is applied
            'from-pos',				#	string	No	connect to (<center>, top, bottom, left, right, top-left, top-right, bottom-left, bottom-right)
            'to-pos',				#	string	No	connect to (<center>, top, bottom, left, right, top-left, top-right, bottom-left, bottom-right)
            'draw-order',			#	int		No	index into draw order relative to parent
            'color',				#	string	No	line color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'style',				#	string	No	line style (none, <solid>, dotted, dashed, dash-dot, dash-dot-dot)
            'thickness',			#	int		No	line thickness (0-6 where 0 = not visible and 6 is the largest)
            'type',					#	string	No	the line type (straight, vector, bezier-3pt, bezier-4pt, or spline)
            'arrowhead',			#	string	No	"how to draw the arrow head on a connection
													#no - never draw the arrow heador ends if bidirectional
													#yes - always draw the arrow head on the end 
													#if-to-concept - draw if ending at a concept
													#if-to-concept-and-slopes-up - draw if ending at a concept and in an upward slope"
            'stylesheet-id',		#	string	No	style info to apply to this connection (default is _default_)
                                                            ])			
        self.dfControlPoints = pd.DataFrame(columns=[		#Control points are used to define how the connection looks. ORDER MATTERS. 
            												#If the connection type is straight, these points are ignored. 
            												#If it is spline or Vector then all points are used. 
            												#If it is bezier-3pt then the first one is used. If it is bezier-4pt the first 2 are used.		
            #'Name',				#	Type	Req	Description
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string	Yes	the unique id of the connection to which this appearance is applied
            'x',					#	int		Yes	x coordinate of the connection control point
            'y',					#	int		Yes	y coordinate of the connection control point
                                                            ])			
        self.dfResourceGrps = pd.DataFrame(columns=[		#List of resource groups in the map.
            #Name                       Type	Req	Description
            'id',					#	string	No	The unique Id of the Connection (required only if appearance is defined)
            'parent-id',			#	string	No	The unique Id of the parent container if not the map (usually the Id of another Node)
                                                            ])
        self.dfResources = pd.DataFrame(columns=[			#Description of a resource linked to by this map.
            #Name						Type	Req	Description
            'id',					#	string	No	the unique id for this resource description (only required if specifying appearance)
            'label',				#	string	Yes	the Text displayed to the viewer if needed
            'description',			#	string	No	a meaningful description of this resource
            'resource-name',		#	string	No	the name of the linked resource
            'resource-mimetype',	#	string	Yes	the MIME type of the linked resource
            'resource-server-id',	#	string	No	The unique Id of the server containing this resource (required if no url specified)
            'resource-folder-id',	#	string	No	The unique Id of the folder containing this resource(required if no url specified)
            'resource-id',			#	string	No	The unique Id of the linked resource (required if no url specified)
            'resource-url',			#	string	No	The url to the linked resource (required if sid,pid,rid triplet not specified)
            'focus-entity-id',		#	string	No	The unique Id of the entity in the specified resource that is to be focused on
                                                            ])
        self.dfResourceApprncs = pd.DataFrame(columns=[		#Defines the individual appearance information for a resource		
            #'Name',				#	Type	Req	Description	
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string	Yes	the unique id of the resource to which this appearance is applied
            'font-name',			#	string	No	the font family's name
            'font-size',			#	decimal	No	the font point size
            'font-style', 			#	string	No	the font style (<plain>|italic|bold|underlined)
            'font-color',			#	string	No	font color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'background-color',		#	string	No	background color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'stylesheet-id',		#	string	No	style info to apply to this node (default is _default_)
                                                            ])			
        self.dfStyleSheets = pd.DataFrame(columns=[			#A Style Sheet contains specific style information about the entities of a map.		
            #'Name',				#	Type	Req	Style Element		Description	
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string	Yes	The unique Id of the Style sheet (_Default_ and _LatestChanges_ are reserved as special ids)
            'name',					#	string	No	the unique name of this style sheet. (_Default_ and _LatestChanges_ are reserved as special names)
            'parent-id',			#	string	No	the parent style sheet id (if a style is not defined here then the parent is checked.)
                                                            ])			
        self.dfStyles = pd.DataFrame(columns=[			#A Style Sheet contains specific style information about the entities of a map.		
            #'Name',				#	Type	Req	Style Element		Description	
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'stylesheet-id',		#	string	Yes	The unique Id of the Style sheet that this row corresponds to
            'name',					#	string	No	the unique name of this style sheet. (_Default_ and _LatestChanges_ are reserved as special names)
            'parent-id',			#	string	No	the parent style sheet id (if a style is not defined here then the parent is checked.)
            'style-type',			#	string	No	the style type defied in this row, one of: [null, map-style, concept-style, linking-phrase-style, connection-style, resorce-style])
            'background-color',		#	string	No	[map, concept, linking phrase, or resource] background color defined in an RGBA value (0-255, 0-255, 0-255, 0 or 255)
            'background-image',		#	string	No	[map, concept, linking phrase] background image id reference. If "none" empty then no background image
            'image-style',			#	string	No	[MAPS ONLY] Image Styles are [not-drawn, full, or tiled]
            'background-image-style',#	string	No	[concept, linking phrase] How to draw the background image of (<full>, scaled, cropped, or tiled)
            'image-top-left',		#	string	No	[MAPS ONLY] Defines the (x,y) position of the top left corner of the background image for a map
            'image-bottom-right',	#	string	No	[MAPS ONLY] Defines the (x,y) position of the bottom-right corner of the background image for a map
            'font-name',			#	string	No	[concept, linking phrase, resource] font family's name
            'font-size',			#	decimal	No	[concept, linking phrase, resource] font point size
            'font-style', 			#	string	No	[concept, linking phrase, resource] font style (<plain>|italic|bold|underlined)
            'font-color',			#	string	No	[concept, linking phrase, resource] font color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'border-color',			#	string	No	[concept, linking phrase] border color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'border-style',			#	string	No	[concept, linking phrase] border line style (none, <solid>, dotted, dashed, dash-dot, dash-dot-dot)
            'border-thickness',		#	int		No	[concept, linking phrase] border line thickness (0-6 where 0 = no border and 6 is the largest)
            'shadow-color',			#	string	No	[concept, linking phrase] shadow color (an RGBA value [0-255, 0-255, 0-255, 0 or 255] or "none" if no shadow) 
            'color',				#	string	No	[CONNECTION ONLY] line color (an RGBA value [0-255, 0-255, 0-255, 0 or 255])
            'style',				#	string	No	[CONNECTION ONLY] line style (none, <solid>, dotted, dashed, dash-dot, dash-dot-dot)
            'thickness',			#	int		No	[CONNECTION ONLY] line thickness (0-6 where 0 = not visible and 6 is the largest)
            'type',					#	string	No	[CONNECTION ONLY] the line type (straight, vector, bezier-3pt, bezier-4pt, or spline)
            'arrowhead',			#	string	No	[CONNECTION ONLY] "how to draw the arrow head on a connection
# 													#yes - always draw the arrow head on the end or ends if bidirectional
# 													#no - never draw the arrow head
# 													#if-to-concept - draw if ending at a concept
# 													#if-to-concept-and-slopes-up - draw if ending at a concept and in an upward slope"
                                                            ])		
        self.dfProperties = pd.DataFrame(columns=[			#For CMAPs, a list of properties for istantiating the <properties-list> items
            												#inside of <extra-properties-list> and <extra-graphical-properties-list>
            #Name						Type	Req	Description
            'mapID',				#	string	No	the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'propertyType'			#	string	No	["extra-property" or "extra-graphical-property"]
            'resource-id',			#	string	No	the resources ID  to which the property belongs eg. ="1W0WVKZ3G-RS5K3L-DS4"
            'key'					#	string	No	the name of the property within the properties list
            'value'					#	string	No	the value of the property key within the properties list
                                                            ])			
        self.dfImages = pd.DataFrame(columns=[				#
            #Name						Type			Req	Description
            'mapID',				#						the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'id',					#	string			Yes	the unique id of this image
            'resource-server-id',	#	string			No	(for CMAPs ONLY) The unique Id of the server containing this resource (required if no url specified)
            'resource-folder-id',	#	string			No	(for CMAPs ONLY) The unique Id of the folder containing this resource(required if no url specified)
            'resource-id',			#	string			No	(for CMAPs ONLY) The unique Id of the linked resource (required if no url specified)
            'resource-url',			#	string			No	The url to the linked resource (required if sid,pid,rid triplet not specified)
            'bytes',				#	base64Binary	Yes	Base 64 encoded image bytes
                                                            ])			
        self.dfPropositions = pd.DataFrame(columns=[ 		#Each proposition is an ordered list of connections that should start and end with a concept with at least one linking phrase in between each concept in the propostion
            #Name	XML Schema Type	Required	Description
            'mapID',				#				the ID of the map for linking dfs/tables: NOT IN CMAPs!
            'conn-id',					#	string	Id of the connection
                                                            ])
        self.dfKGintrfcs = pd.DataFrame(columns=[ 		#
            #Name					#	Type	Description
            'namDF',				#	string	the  name of the df!
            'namIntrfc'				#	string	the name of the interface (e.g. cxl, ttl, d3, etc)
            'namRootDic',			# 	string	PKG rootDic Name (where parents live)
            'valRootDic',			# 	string	PKG rootDic Value
            'locDic',				#	string	the location of the PKG subDic 
            'namNodParent',			#	string	the name of the nodParent e.g. 'concept-list'
            'namNodFamily',			#	string	the name of the nod member (e.g. 'concept')
                                                            ])
        data = {
            'namDF':[self.dfResMetaList,self.dfPartList,self.dfParts,self.dfResMetaSansPeople,self.dfResMetaPeople,
                           self.dfMap,self.dfConcepts,self.dfConceptApprncs,self.dfLinkingPhrases,self.dfLinkingPhraseApprncs,
                           self.dfLocalizedStyles,self.dfConnections,self.dfConnectionApprncs,self.dfControlPoints,
                           self.dfResourceGrps,self.dfResources,self.dfResourceApprncs,self.dfStyleSheets,self.dfStyles,
                           self.dfProperties,self.dfImages,self.dfPropositions
                    ],
            'namIntrfc':['cxl'],
            'namRootDic':[np.NaN,np.NaN,np.NaN,self.rootDicName,self.rootDicName,self.rootDicName,self.rootDicName,self.rootDicName,
                           self.rootDicName,self.rootDicName,self.rootDicName,self.rootDicName,self.rootDicName,
                           self.rootDicName,self.rootDicName,self.rootDicName,self.rootDicName,self.rootDicName,
                           self.rootDicName,self.rootDicName,self.rootDicName,self.rootDicName
                          ],
            'valRootDic':[np.NaN,np.NaN,np.NaN,self.rootDicVal,self.rootDicVal,self.rootDicVal,self.rootDicVal,self.rootDicVal,
                          self.rootDicVal,self.rootDicVal,self.rootDicVal,self.rootDicVal,self.rootDicVal,
                          self.rootDicVal,self.rootDicVal,self.rootDicVal,self.rootDicVal,self.rootDicVal,
                          self.rootDicVal,self.rootDicVal,self.rootDicVal,self.rootDicVal
                         ],
            'dicLoc':[np.NaN,np.NaN,np.NaN,self.dicResMetaSansPeople,self.dicResMetaPeople,self.dicMap,self.dicMap,self.dicMap,\
                      self.dicMap,self.dicMap,self.dicMap,self.dicMap,self.dicMap,self.dicMap,self.dicMap,
                      self.dicMap['resource-group-list']['resource-group'],self.dicMap['resource-group-list']['resource-group'],
                      self.dicMap['style-sheet-list']['style-sheet'],self.dicMap['style-sheet-list']['style-sheet'],
                      self.dicMap,self.dicMap,self.dicMap
                     ],
            'namNodParent':[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,'concept-list','concept-appearance-list','linking-phrase-list',
                         'linking-phrase-appearance-list',self.listNodeApprncs,
                         'connection-list','connection-appearance-list',self.listResMetaPeopleOrgs,'resource-group-list',
                         self.itemCounter,'resource-appearance-list','style-sheet-list',self.itemCounter,
                         [self.dicMap['extra-property-list'],self.dicMap['extra-graphical-property-list']],np.NaN,np.NaN
                        ],
            'namNodFamily':[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,'map','concept','concept-appearance','linking-phrase','linking-phrase-appearance'
                        ]
                }
        
        #Test for arguments passed into CMAP, and parse as necessary 
        if cxlDoc is not None:
            self.dicDoc = xmltodict.parse (cxlDoc)
        if cmapID is not None:
            self.cmapID = cmapID
            cxlDoc = self.getCMAP()
    def getCMAP (self):
        """
        Given a CMAP ID and possibly a Server URL, read a CMAP from a CMAP server, the default server being the CMAPs cloud server
        Note we are using this webservice: https://cmap.ihmc.us/xml/Cmapserver-HTTP-API.pdf
        TODO:
        1) make a list of all cmap entities, cycle throgh it, and make dfs
        """
        import getpass
        import xmltodict
        
        #For CMaps GET commands: get.resmeta, get.cmap
        self.getMetacom = '/?cmd=get.resmeta' 
        self.getCMAPcom = '/?cmd=get.cmap' 
        self.getPermscom = '/?cmd=get.permissions' 
        self.url = self.baseURL + self.cmapID + self.getCMAPcom
        self.pwd = getpass.getpass(prompt='Password: ', stream=None) 
        self.r = requests.get(self.url, auth=(self.authEmail, self.pwd), allow_redirects=True, verify=True)
        self.cxlDoc = self.r.text
        self.dicDoc = xmltodict.parse (self.cxlDoc) 
        self.mapID = self.dicDoc['cmap']['res-meta']['dc:source'].split(':')[-1]
        self.folderID = self.dicDoc['cmap']['res-meta']['dc:source'].split(':')[-2]
        self.rootDicName = 'mapID'
        self.rootDicVal = self.mapID    #we'll use rootDicVal as we parse the dics
        self.listResMetaPeopleOrgs = ['dc:creator', 'dc:contributor', 'dcterms:rightsHolder',]
        self.dicMapResMeta = self.dicDoc['cmap']['res-meta'] #with people
        self.dicResMetaSansPeople = {key:val for key, val in dict(self.dicDoc['cmap']['res-meta']).items() if key not in self.listResMetaPeopleOrgs}
        self.dicResMetaPeople = {key:val for key, val in dict(self.dicDoc['cmap']['res-meta']).items() if key in self.listResMetaPeopleOrgs}
        self.dicMap = self.dicDoc['cmap']['map']
    def countDicItems (self,d):
        """
        takes a (sub) dictionary and returns the number of items (lists or dics)
        """
        count = 0
        for x in enumerate(d.items()): 
            if isinstance(x[1][1], list): 
                count += len(x[1][1]) 
        return (count)
    def subDic2DF (self, rootDicName, rootDicVal, dicLoc, nodParent, nodFamily, dicColRenames={}):
        """
        takes a parent (sub) dicionary (node) and generates a df from attributes of either: 
        *heterogenous child nodes (if family is NOT specified)
        *homogeneous child nodes (if node family IS SPECIFIED)
        to rename columns  dicColRenames provided in form of {dicNodeName:dfColName}--DEPRECATED, to be removed--use external library
        """
        
        if (isinstance(nodParent, int)) or \
            (nodParent is None) or \
            (nodFamily is None) or \
            (nodParent in dicLoc): 
            if nodParent is None:
                dicParent = dicLoc
            else:
                dicParent = dicLoc[nodParent]
                #print(dicParent)
                nodeCount  = self.countDicItems(dicParent)
                #print('nodeCount:' + str(nodeCount))
                keyCount = len(dicParent)
            #print('keyCount:' + str(keyCount))
            if nodFamily is None: #make a df for the keys at this level
                keyCount = len(dicParent)
                if keyCount>0:
                    pass #print('')
                df = pd.DataFrame([dicParent])
            else:
                if nodeCount>1:
                    df = pd.DataFrame(dicParent[nodFamily]) # turn dic into dataframe
                else:
                    df = pd.DataFrame([dicParent[nodFamily]] )#accomodate singleton as a non-scalar with extra '[]'
                df.rename(columns= dicColRenames,inplace=True)    #get rid of the dict names and make more SQL friendly
            for column in df: #rename columns to remove/replace special characters
                ampNum = str(column).find('@')  #eventually update this to find all special characters that may be in a dict name
                if ampNum >-1:
                    newCol = str(column)[ampNum+1:]
                    df.rename(columns= {column:newCol},inplace=True)
                newCol = str(column).replace(':',':')
                df.rename(columns= {column:newCol},inplace=True)
            df.insert(loc=0,column=rootDicName, value=rootDicVal) # add rootID as a column   
            return(df)
    def dfToXML (self, df=None, xmlBranch='', xmlName='', xmlIorA = 'A', mode='w'):
        """
        Turns a dataframe into an xml branch with either items (xmlIorA flag = 'I')
        or attributes (xmlIorA flag = 'A')
        """
        pulNans = ',\s+[^,]+=nan'
        xml=['<'+xmlBranch+'>']
        df.fillna('', inplace=True)
        if xmlIorA == 'I': #Columns in df represent items in a branch of the xml tree.
            def row_to_xml_I(row):
                xml = ['<' + xmlBranch + '>']
                for i, colName in enumerate(row.index):
                    if str(row.iloc[i])!='':
                        xml.append('    <' + colName + '>'+ str(row.iloc[i]).replace('\n','&#xa;') + '</' + colName + '>')
                xml.append('</' + xmlBranch + '>')
                return '\n'.join(xml)
            res = '\n'.join(df.apply(row_to_xml_I, axis=1))
            return (res)
        else: #Columns in the df represent attributes in one or more xml nodes that sit within a branch pof the xml tree
            strBranchBeg =  '<' + xmlBranch + '>'
            strBranchEnd =  '</' + xmlBranch + '>'
            def row_to_xml_A(row):
                xml =[]
                preRowCol = '    <' + xmlName + '> '
                postRowCol = '</' + xmlName + '>'
                rowText = ''
                for i, colName in enumerate(row.index):
                    if str(row.iloc[i])!='':
                        rowText += ' '+ colName + '='+ '"' + str(row.iloc[i]).replace('\n','&#xa;')+'"' 
                xml.append(preRowCol + rowText + postRowCol)
                return '\n'.join(xml)
            res = '\n'.join(df.apply(row_to_xml_A, axis=1))
            res = '\n'.join([strBranchBeg, res ,strBranchEnd])
            return (res)
    def getResMetaSansPeople(self):
        """Make a df of the map (resource) metadata sans people (they're in another df"""
        if len(self.dfResMetaSansPeople)==0: #will not generate a new df if there are already rows in it
            Dic2DFArgs = {
            'rootDicName':self.rootDicName, 
            'rootDicVal':self.rootDicVal, 
            'dicLoc':self.dicResMetaSansPeople, 
            'nodParent':None, 
            'nodFamily':None
                    }
            self.dfResMetaSansPeople = self.dfResMetaSansPeople.append(self.subDic2DF(**Dic2DFArgs))
        return (self.dfResMetaSansPeople)
    def genResMetaSansPeopleXML(self):
        df = self.getResMetaSansPeople()
        dfToXMLargs = {
            'df':df.iloc[:,1:], #skip the mapID columnsince it isn't in the cxl spec
            'xmlBranch':'res-meta',
            'xmlName':'', 
            'xmlIorA':'I',
                    }
        return (self.dfToXML(**dfToXMLargs))
    def getResMetaPeople(self):
        """append df of all the people/roles/orgs sans other metadata from cxl into self.df """
        if len(self.dfResMetaPeople)==0: #will not generate a new df if there are already rows in it
            Dic2DFArgs = {
            'rootDicName':self.rootDicName, 
            'rootDicVal': self.mapID, 
            'dicLoc': self.dicResMetaPeople, 
            'nodParent': None, 
            'nodFamily': None
                }
            for item in self.listResMetaPeopleOrgs: #Need to cycle through each of the roles to make entries
                                                    #Will need to add logic here to create entries for 
                                                    #creators/contributors/rightsHolders of indiv concepts/propositions
                    Dic2DFArgs["nodParent"] = item
                    self.dfResMetaPeople = self.dfResMetaPeople.append(self.subDic2DF(**Dic2DFArgs))
                    self.dfResMetaPeople['role'].iat[-1] =item.split(':')[-1]
                    self.dfResMetaPeople['vcard:ORG'].iat[-1] = str(self.dfResMetaPeople['vcard:ORG'].iat[-1]).split(", '",1)[-1][:-4] 
        return(self.dfResMetaPeople)
    def genResMetaPeopleXML(self):
        """make xml of all the people/orgs responsible for CMAP"""
        df = self.getResMetaPeople()
        dfToXMLargs = {
            'df':df,
            'xmlBranch':'',
            'xmlName':'', 
            'xmlIorA':'I',
                    }
        peopleXML = ''
        for item in self.listResMetaPeopleOrgs: #Cycle through each of the roles to make entries
                                                #need to logic for creators/contributors/rightsHolders of indiv concepts/propositions
                                                #<vcard:ORG> section is a cludge and needs proper fixing
            dfToXMLargs["xmlBranch"] = item
            dfToXMLargs["df"]= df[(df.role == item.split(':')[-1])].iloc[:,4:7]
            peopleXML = '\n'.join([peopleXML,self.dfToXML(**dfToXMLargs)])
        peopleXML = peopleXML\
                            .replace('<vcard:ORG>','\t<vcard:ORG>\n\t\t<vcard:Orgname>')\
                            .replace('</vcard:ORG>','</vcard:Orgname>\n\t</vcard:ORG>')\
                            .replace('\n','\n    ')
            #print(dfToXMLargs["xmlBranch"])
        return(peopleXML)
    def genResMetaXML(self): #Use this for a complete res-meta section
        """make a df of all the resource metadata in the CMAP"""
        resMetaXML = '\n'.join([self.genResMetaSansPeopleXML().rsplit("\n",1)[0], \
                               #self.genResMetaPeopleXML().split("\n",1)[1],\
                               self.genResMetaPeopleXML().rsplit("\n",1)[1]])
        return(resMetaXML)
    def getConceptList(self):
        """make a df of all the concepts in the CMAP"""
        if len(self.dfConcepts)==0: #will not generate a new df if there are already rows in it
            Dic2DFArgs = {
            'rootDicName':self.rootDicName, 
            'rootDicVal': self.mapID, 
            'dicLoc': self.dicMap, 
            'nodParent': 'concept-list', 
            'nodFamily': 'concept'
                }
            self.dfConcepts = self.subDic2DF(**Dic2DFArgs)
        return (self.dfConcepts)
    def genConceptListXML(self):
        """
        need to change XML parsing logic to:
            get cxlBranch from nodParent 
            get cxlName from nodName
            in dfPKGintrfcs
        then change arguments/params to read dfPKGintrfcs rows filtering on submitted dfName 
        store list of column names that should be extracted from given df for specific interface
        """
        df = self.getConceptList()
        dfToXMLargs = {
            'df': df.iloc[:,1:],
            'xmlBranch':'concept-list',
            'xmlName':'concept', 
            'xmlIorA':'A'
                    }
        return (self.dfToXML(**dfToXMLargs))
    def getConceptAppearanceList(self):
        """a df of the ConceptAppearances"""
        if len(self.dfConceptApprncs)==0: #will not generate a new df if there are already rows in it
            Dic2DFArgs = {
                'rootDicName':self.rootDicName, 
                'rootDicVal': self.mapID, 
                'dicLoc': self.dicMap, 
                'nodParent' : 'concept-appearance-list',
                'nodFamily' : 'concept-appearance'
                    }
            self.dfConceptApprncs = self.subDic2DF(**Dic2DFArgs)
        return(self.dfConceptApprncs)
    def genConceptAppearanceListXML(self):
        df = self.getConceptAppearanceList()
        dfToXMLargs = {
            'df': df.iloc[:,1:],
            'xmlBranch':'concept-appearance-list',
            'xmlName':'concept-appearance', 
            'xmlIorA':'A'
                    }
        return (self.dfToXML(**dfToXMLargs))
    def getLinkingPhraseList(self):
        """make a df of all the concepts in the CMAP"""
        if len(self.dfLinkingPhrases)==0: #will not generate a new df if there are already rows in it
            Dic2DFArgs = {
            'rootDicName':self.rootDicName, 
            'rootDicVal': self.mapID, 
            'dicLoc': self.dicMap, 
            'nodParent': 'linking-phrase-list', 
            'nodFamily': 'linking-phrase'
                }
            self.dfLinkingPhrases = self.subDic2DF(**Dic2DFArgs)
        return (self.dfLinkingPhrases)

    def genLinkingPhraseListXML(self):
        df = self.getLinkingPhraseList()
        dfToXMLargs = {
            'df': df.iloc[:,1:],
            'xmlBranch':'linking-phrase-list',
            'xmlName':'linking-phrase', 
            'xmlIorA':'A'
                    }
        return (self.dfToXML(**dfToXMLargs))
    
    def getLinkingPhraseAppearanceList(self):
        """a df of the Linking Phrase Appearances"""
        if len(self.dfLinkingPhraseApprncs)==0: #will not generate a new df if there are already rows in it
            Dic2DFArgs = {
                'rootDicName':self.rootDicName, 
                'rootDicVal': self.mapID, 
                'dicLoc': self.dicMap, 
                'nodParent' : 'linking-phrase-appearance-list',
                'nodFamily' : 'linking-phrase-appearance'
                    }
            self.dfLinkingPhraseApprncs = self.subDic2DF(**Dic2DFArgs)
        return(self.dfLinkingPhraseApprncs)  
    
    def genLinkingPhraseAppearanceListXML(self):
        df = self.getLinkingPhraseList()
        dfToXMLargs = {
            'df': df.iloc[:,1:],
            'xmlBranch':'linking-phrase-appearance-list',
            'xmlName':'linking-phrase-appearance', 
            'xmlIorA':'A'
                    }
        return (self.dfToXML(**dfToXMLargs))   

In [42]:
myMap = CMAP(cmapID='1QBMKWB9H-P56N1M-6XJ') # '1W46SJ7J7-CZ2M5Y-3MW')

Password: ········


In [48]:
#myMap.resMetaPeopleDF()
#print(myMap.dfResMetaPeople)
# df= myMap.getResMetaPeople().iloc[:,3:7]
# display(df)
# newdf = df[(df.role == 'dc:creator'.split(':')[-1])].iloc[:,1:4]
# display(newdf)
#display(df)
#print(myMap.genLinkingPhraseListXML())
# display(myMap.dfConcepts.iloc[:,1:])
print("ResMeta:  " myMap.genResMetaXML())
display(myMap.getConceptList())
print(myMap.genConceptListXML())
display(myMap.getConceptAppearanceList())
display(myMap.getLinkingPhraseList())
display(myMap.getLinkingPhraseAppearanceList())


SyntaxError: invalid syntax (3872173810.py, line 10)